In [ ]:
%pip install -qU pytorch-partial-tagger

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.6 MB/s eta 0:00:00


# Download datasets

Here, we will download datasets provided on [teffland/ner-expected-entity-ratio](https://github.com/teffland/ner-expected-entity-ratio/tree/main) below. We use the datasets for the experimental setting Non-Native Speaker Scenario (NNS): Recall=50%, Precision=90% in [Effland and Collins. (2021)](https://aclanthology.org/2021.tacl-1.78/).


In [ ]:
!curl -LO https://raw.githubusercontent.com/teffland/ner-expected-entity-ratio/main/data/conll2003/eng/entity.train_r0.5_p0.9.jsonl
!curl -LO https://raw.githubusercontent.com/teffland/ner-expected-entity-ratio/main/data/conll2003/eng/entity.dev.jsonl
!curl -LO https://raw.githubusercontent.com/teffland/ner-expected-entity-ratio/main/data/conll2003/eng/entity.test.jsonl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3918k  100 3918k    0     0  5866k      0 --:--:-- --:--:-- --:--:-- 5857k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1165k  100 1165k    0     0  1841k      0 --:--:-- --:--:-- --:--:-- 1838k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1114k  100 1114k    0     0  2858k      0 --:--:-- --:--:-- --:--:-- 2858k


# Import all dependencies

In [ ]:
import json
import logging
import random
import sys
from contextlib import contextmanager

import numpy as np
import torch

from partial_tagger.metric import Metric
from partial_tagger.utils import create_tag, create_trainer

# Define utility functions

We will prepare two utility functions. One is for fixing random state and the other is for displaying training logs.

In [ ]:
def fix_state(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


class JSONAdapter(logging.LoggerAdapter):
    def process(self, msg, kwargs):
        return json.dumps(msg), kwargs


@contextmanager
def get_logger(log_name, log_file):
    logger = logging.getLogger(log_name)
    logger.propagate = False

    logger.setLevel(logging.INFO)

    handlers = [
        logging.StreamHandler(sys.stdout),
        logging.FileHandler(log_file, mode="w", encoding="utf-8"),
    ]

    for handler in handlers:
        logger.addHandler(handler)

    try:
        yield JSONAdapter(logger, {})
    finally:
        for handler in handlers:
            handler.close()
            logger.removeHandler(handler)
        logging.shutdown()

# Prepare datasets

We will prepare our datasets. Each item of dataset must have a pair of a string and tags. A string represents `text` below. Tags represent a hash set of character-based tags, where each tag has a start, a length, and a label, which are defined as `tags` below. A start represents a position in text where a tag starts. A length represents a distance in text between the beginning of a tag and the end of a tag. A label represents what you want to assign to a span of text defined by a start and a length.

In [ ]:
def load_dataset(path: str):
    with open(path) as f:
        dataset = []

        for line in f:
            data = json.loads(line.rstrip())

            text = " ".join(data["tokens"])

            mapping = {}
            now = 0
            for i, token in enumerate(data["tokens"]):
                mapping[i] = now
                now += len(token) + 1  # Add one for a space

            tags = {
                create_tag(
                    mapping[annotation["start"]],
                    len(annotation["mention"]),
                    annotation["type"],
                )
                for annotation in data["gold_annotations"]
            }

            dataset.append((text, tags))

    return dataset


train_dataset = load_dataset("entity.train_r0.5_p0.9.jsonl")
dev_dataset = load_dataset("entity.dev.jsonl")
test_dataset = load_dataset("entity.test.jsonl")

# Train our tagger

We will train our tagger by creating an instance of `Trainer` and passing datasets to it. After training, `trainer` gives you an instance of `Recognizer` which predicts character-based tags from given texts.

In [ ]:
seed = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "roberta-base"
dropout = 0.2
batch_size = 15
num_epochs = 20
learning_rate = 2e-5
gradient_clip_value = 5.0
padding_index = -1
unknown_index = -100
train_log_file = "log.jsonl"
tokenizer_args = {
    "padding": True,
    "return_tensors": "pt",
    "return_offsets_mapping": True
}

fix_state(seed)

trainer = create_trainer(
    model_name=model_name,
    dropout=dropout,
    batch_size=batch_size,
    num_epochs=num_epochs,
    learning_rate=learning_rate,
    gradient_clip_value=gradient_clip_value,
    padding_index=padding_index,
    encoder_type="default",
)

with get_logger(f"{__name__}.train", train_log_file) as logger:
  recognizer = trainer(
      train_dataset,
      dev_dataset,
      device,
      logger
  )

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{"epoch": 1, "loss": 95725.91049556434, "validation_f1_score": 0.8765173712850565, "validation_precision": 0.872063968015992, "validation_recall": 0.8810164927633793}
{"epoch": 2, "loss": 37851.56417286815, "validation_f1_score": 0.9287749287749287, "validation_precision": 0.9248998664886515, "validation_recall": 0.9326825984516998}
{"epoch": 3, "loss": 31892.290308916126, "validation_f1_score": 0.9297143335846527, "validation_precision": 0.925604670558799, "validation_recall": 0.9338606529787951}
{"epoch": 4, "loss": 27859.101005028002, "validation_f1_score": 0.9304010785305021, "validation_precision": 0.9316571042861964, "validation_recall": 0.929148434870414}
{"epoch": 5, "loss": 25275.790772246895, "validation_f1_score": 0.9277322288902037, "validation_precision": 0.9319069451519782, "validation_recall": 0.9235947492426793}
{"epoch": 6, "loss": 22689.852550179407, "validation_f1_score": 0.9144174923129484, "validation_precision": 0.9283732223378425, "validation_recall": 0.900875126

# Evalute our tagger

We will evaluate the performance of our tagger using Metric as follows.

In [ ]:
texts, ground_truths = zip(*test_dataset)

predictions = recognizer(texts, batch_size, device)

metric = Metric()
metric(predictions, ground_truths)

test_scores_file = "scores.json"
with get_logger(f"{__name__}.evaluate", test_scores_file) as logger:
  logger.info({f"test_{key}": value for key, value in metric.get_scores().items()})

{"test_f1_score": 0.8929986789960369, "test_precision": 0.8883826879271071, "test_recall": 0.8976628895184136}
